# Welcome to the Land of Generative AI and Mathematical Optimization on Gurobi!

## Our goal...
We will be revisiting the casino chip production problem, but this time, we will leverage our Gurobot assistant. In the beginning, we will simply start with the basic problem setup for the casino chip production problem. From there, we will assess the problem in increasing complexity:

## Challenges
- Help me define the basic problem
- Let's adjust the objective function equation
- How about three new business constraints?

## Extra Challenges
- Moving to a single factory to three casino expansion
- Moving from fixed demand to variable and logarithmic demand (nonlinearity)
- Extending to three factories to three casinos
- Introducing transportation cost balancing

## Tools you can use:
- [Gurobot](https://portal.gurobi.com/iam/chat)
    - NOTE: You will need to create an account
- Gurobo AI Modeling
    - [Prompt Engineer](https://chatgpt.com/g/g-JK2EuyVOt-gurobi-ai-modeling-prompt-engineer)
    - [Modeling Assistant](https://chatgpt.com/g/g-g69cy3XAp-gurobi-ai-modeling-assistant)
- Any other LLM you want!

## Chat Flow

### Prompt 1

You are a data scientist tasked with optimizing the poker chip production at a casino. Management would like to make these new chips using the on-hand inventory of raw material. Right now, you are asked to manufacture the highest possible total value in poker chips. The denominations of chips are \\$1, \\$5, \\$10, \\$25, \\$100, \\$500, and \\$1000. Each denomination of chip requires a different amount of several raw materials.

**Objective:**  
Maximize total value of chips

**Constraints:**

- **Material Amount Constraint:** There are limited amounts of materials available across all poker chip types. 

The data recipe is the following:
```python
import gurobipy as gp

ingredients, on_hand = gp.multidict(
    {
        "clay":      5000,
        "lead":      1500,
        "silver":     500,
        "gold":        50,
    }
)

recipes = { 
        ("one",          "clay"):  18.0, ("one",          "lead"):  0.0, ("one",          "silver"):  0, ("one",          "gold"):  0, 
        ("five",         "clay"):  16.0, ("five",         "lead"):  1.0, ("five",         "silver"):  0, ("five",         "gold"):  0, 
        ("ten",          "clay"):  15.0, ("ten",          "lead"):  2.0, ("ten",          "silver"):  0, ("ten",          "gold"):  0, 
        ("twenty-five",  "clay"):  13.0, ("twenty-five",  "lead"):  4.5, ("twenty-five",  "silver"):  0, ("twenty-five",  "gold"):  0, 
        ("one hundred",  "clay"):  10.0, ("one hundred",  "lead"):  6.0, ("one hundred",  "silver"):  1, ("one hundred",  "gold"):  0, 
        ("five hundred", "clay"):  10.0, ("five hundred", "lead"):  8.5, ("five hundred", "silver"):  2, ("five hundred", "gold"):  0, 
        ("thousand",     "clay"):  10.0, ("thousand",     "lead"):  9.5, ("thousand",     "silver"):  0, ("thousand",     "gold"):  2, 
}
```


>Optional: The following is the data recipe using gurobipy and pandas.

In [ ]:
import gurobipy as gp
import pandas as pd

# Convert ingredients to pandas dictionary
ingredients_df = pd.DataFrame([
    {"ingredient": "clay", "on_hand": 5000},
    {"ingredient": "lead", "on_hand": 1500},
    {"ingredient": "silver", "on_hand": 500},
    {"ingredient": "gold", "on_hand": 50}
])
ingredients, on_hand = gp.multidict(ingredients_df.set_index("ingredient")["on_hand"].to_dict())

# Convert recipes to pandas dictionary
recipes_data = [
    {"chip": "one", "ingredient": "clay", "amount": 18.0},
    {"chip": "one", "ingredient": "lead", "amount": 0.0},
    {"chip": "one", "ingredient": "silver", "amount": 0},
    {"chip": "one", "ingredient": "gold", "amount": 0},
    {"chip": "five", "ingredient": "clay", "amount": 16.0},
    {"chip": "five", "ingredient": "lead", "amount": 1.0},
    {"chip": "five", "ingredient": "silver", "amount": 0},
    {"chip": "five", "ingredient": "gold", "amount": 0},
    {"chip": "ten", "ingredient": "clay", "amount": 15.0},
    {"chip": "ten", "ingredient": "lead", "amount": 2.0},
    {"chip": "ten", "ingredient": "silver", "amount": 0},
    {"chip": "ten", "ingredient": "gold", "amount": 0},
    {"chip": "twenty-five", "ingredient": "clay", "amount": 13.0},
    {"chip": "twenty-five", "ingredient": "lead", "amount": 4.5},
    {"chip": "twenty-five", "ingredient": "silver", "amount": 0},
    {"chip": "twenty-five", "ingredient": "gold", "amount": 0},
    {"chip": "one hundred", "ingredient": "clay", "amount": 10.0},
    {"chip": "one hundred", "ingredient": "lead", "amount": 6.0},
    {"chip": "one hundred", "ingredient": "silver", "amount": 1},
    {"chip": "one hundred", "ingredient": "gold", "amount": 0},
    {"chip": "five hundred", "ingredient": "clay", "amount": 10.0},
    {"chip": "five hundred", "ingredient": "lead", "amount": 8.5},
    {"chip": "five hundred", "ingredient": "silver", "amount": 2},
    {"chip": "five hundred", "ingredient": "gold", "amount": 0},
    {"chip": "thousand", "ingredient": "clay", "amount": 10.0},
    {"chip": "thousand", "ingredient": "lead", "amount": 9.5},
    {"chip": "thousand", "ingredient": "silver", "amount": 0},
    {"chip": "thousand", "ingredient": "gold", "amount": 2}
]

recipes_df = pd.DataFrame(recipes_data)
recipes = recipes_df.set_index(["chip", "ingredient"])["amount"].to_dict()

### Prompt 2

Instead of maximizing the total chip value, we are asked to maximize the total number of chips made. Write out this objective, and modify the previous responses to accommodate this new objective.

### Prompt 3

I have received a new set of constraints from management. Here are they are:
- The value produced by high value chips cannot exceed the 50% of the value produced by low value chips.
- Each of the of low value chips must be at least 15% of the total chips made and each of the high value chips cannot be more than 25%. 
- The \$25 chip is our most used; make sure that it has the maximum number among all chips.

Please modify the code to accommodate the above three new constraints.

> NOTE ONLY (DO NOT INCLUDE) - Gurobot may prompt user for clarity on the definition of high value and low value chips.
>
>At this point in the chatflow, there is a possibility that the context window could be reached; therefore, it may be prudent to potentially begin a new chat with truncated code that contains just the model build.

### Prompt 4

Given the following Gurobi Python optimization program...

```python
def build_model(self):
    """Build the optimization model"""
    with self.env.start():
        self.model = gp.Model("PokerChipProduction_WithNewConstraints", env=self.env)
        # Decision variables: number of chips to produce for each type
        self.x = self.model.addVars(self.chips, vtype=GRB.CONTINUOUS, name="chips")
        # Original material constraints
        self.model.addConstrs(
            (
                gp.quicksum(
                    self.recipes.get((chip, ingredient), 0) * self.x[chip]
                    for chip in self.chips
                )
                <= self.on_hand[ingredient]
                for ingredient in self.ingredients
            ),
            name="material_constraint",
        )
        # NEW CONSTRAINT 1: Value produced by high value chips <= 50% of value produced by low value chips
        high_value_total = gp.quicksum(
            self.values[chip] * self.x[chip] for chip in self.high_value_chips
        )
        low_value_total = gp.quicksum(
            self.values[chip] * self.x[chip] for chip in self.low_value_chips
        )
        self.model.addConstr(
            high_value_total <= 0.5 * low_value_total, name="high_vs_low_value"
        )
        # Total chips variable for percentage constraints
        total_chips = gp.quicksum(self.x[chip] for chip in self.chips)
        # NEW CONSTRAINT 2a: Each low value chip must be at least 15% of total chips
        self.model.addConstrs(
            (self.x[chip] >= 0.15 * total_chips for chip in self.low_value_chips),
            name="low_value_min_15pct",
        )
        # NEW CONSTRAINT 2b: Each high value chip cannot be more than 25% of total chips
        self.model.addConstrs(
            (self.x[chip] <= 0.25 * total_chips for chip in self.high_value_chips),
            name="high_value_max_25pct",
        )
        # NEW CONSTRAINT 3: $25 chip has maximum number among all chips
        self.model.addConstrs(
            (
                self.x["twenty-five"] >= self.x[chip]
                for chip in self.chips
                if chip != "twenty-five"
            ),
            name="twenty_five_maximum",
        )
        # Objective: maximize total number of chips
        self.model.setObjective(
            gp.quicksum(self.x[chip] for chip in self.chips), GRB.MAXIMIZE
        )
```

…management now wants to take into account that there is one single factory supplying chips for multiple casinos. Assume that there are 10 casinos each with unique fixed demands per chip denomination. Please create hypothetical fixed demand numbers. Finally, execute the new program and show logs.

### Prompt 5

Now please express the fixed demand constants from each of the three casinos per chip as decision variables contained in a natural logarithm. In this way, we expect that demand to have increasingly diminishing gains. All other parts of the model and code shall remain the same. 

Here is the following Gurobi Python optimization program to use...

```python
def build_model(self):
    """Build the optimization model"""
    with self.env.start():
        self.model = gp.Model("PokerChipProduction_WithNewConstraints", env=self.env)
        # Decision variables: number of chips to produce for each type
        self.x = self.model.addVars(self.chips, vtype=GRB.CONTINUOUS, name="chips")
        # Original material constraints
        self.model.addConstrs(
            (
                gp.quicksum(
                    self.recipes.get((chip, ingredient), 0) * self.x[chip]
                    for chip in self.chips
                )
                <= self.on_hand[ingredient]
                for ingredient in self.ingredients
            ),
            name="material_constraint",
        )
        # NEW CONSTRAINT 1: Value produced by high value chips <= 50% of value produced by low value chips
        high_value_total = gp.quicksum(
            self.values[chip] * self.x[chip] for chip in self.high_value_chips
        )
        low_value_total = gp.quicksum(
            self.values[chip] * self.x[chip] for chip in self.low_value_chips
        )
        self.model.addConstr(
            high_value_total <= 0.5 * low_value_total, name="high_vs_low_value"
        )
        # Total chips variable for percentage constraints
        total_chips = gp.quicksum(self.x[chip] for chip in self.chips)
        # NEW CONSTRAINT 2a: Each low value chip must be at least 15% of total chips
        self.model.addConstrs(
            (self.x[chip] >= 0.15 * total_chips for chip in self.low_value_chips),
            name="low_value_min_15pct",
        )
        # NEW CONSTRAINT 2b: Each high value chip cannot be more than 25% of total chips
        self.model.addConstrs(
            (self.x[chip] <= 0.25 * total_chips for chip in self.high_value_chips),
            name="high_value_max_25pct",
        )
        # NEW CONSTRAINT 3: $25 chip has maximum number among all chips
        self.model.addConstrs(
            (
                self.x["twenty-five"] >= self.x[chip]
                for chip in self.chips
                if chip != "twenty-five"
            ),
            name="twenty_five_maximum",
        )
        # Objective: maximize total number of chips
        self.model.setObjective(
            gp.quicksum(self.x[chip] for chip in self.chips), GRB.MAXIMIZE
        )
```

Finally, execute the new program and show logs.

>There is a possibility that Gurobot may respond that it does not have enough concrete data to continue on. In that case, the user may prompt Gurobot to infer the underlying data.

### Prompt 6

Please take the model, extend this by accommodating production from three factories (instead of one) to three casinos.

Here is the following Gurobi Python optimization program to use...

```python
    def build_model(self):
        """Build the optimization model for multiple casinos with logarithmic demand objective"""
        with self.env.start():
            self.model = gp.Model("PokerChipProduction_MultiCasino_LogDemand", env=self.env)
            
            # Decision variables: number of chips to produce for each type and casino
            self.x = self.model.addVars(self.casinos, self.chips, vtype=GRB.CONTINUOUS, 
                                      name="chips", lb=0)
            
            # Logarithmic variables for the objective function
            # We'll use log(production + min_demand) to ensure positive arguments
            self.log_vars = self.model.addVars(self.casinos, self.chips, vtype=GRB.CONTINUOUS,
                                             name="log_production", lb=-GRB.INFINITY)
            
            # Add logarithmic constraints: log_vars[casino, chip] = ln(x[casino, chip] + min_demands[casino, chip])
            for casino in self.casinos:
                for chip in self.chips:
                    # Create auxiliary variable for x + min_demand
                    aux_var = self.model.addVar(vtype=GRB.CONTINUOUS, name=f"aux_{casino}_{chip}", 
                                              lb=self.min_demands[casino, chip])
                    
                    # aux_var = x + min_demand
                    self.model.addConstr(aux_var == self.x[casino, chip] + self.min_demands[casino, chip],
                                       name=f"aux_def_{casino}_{chip}")
                    
                    # log_vars = ln(aux_var)
                    self.model.addGenConstrLog(aux_var, self.log_vars[casino, chip], 
                                             name=f"log_constr_{casino}_{chip}")
            
            # Material constraints: total production across all casinos cannot exceed available materials
            self.model.addConstrs(
                (gp.quicksum(self.recipes.get((chip, ingredient), 0) * self.x[casino, chip] 
                           for casino in self.casinos for chip in self.chips) <= self.on_hand[ingredient] 
                 for ingredient in self.ingredients), 
                name="material_constraint"
            )
            
            # Minimum demand constraints: must meet at least minimum demand for each casino and chip type
            self.model.addConstrs(
                (self.x[casino, chip] >= self.min_demands[casino, chip] 
                 for casino in self.casinos for chip in self.chips),
                name="min_demand_constraint"
            )
            
            # For each casino, apply modified business constraints (more relaxed)
            for casino in self.casinos:
                # Total chips variable for percentage constraints (per casino)
                total_chips = gp.quicksum(self.x[casino, chip] for chip in self.chips)
                
                # Modified constraint: Each low value chip must be at least 10% of total chips (reduced from 15%)
                self.model.addConstrs(
                    (self.x[casino, chip] >= 0.10 * total_chips for chip in self.low_value_chips),
                    name=f"low_value_min_10pct_{casino}"
                )
                
                # Modified constraint: Each high value chip cannot be more than 35% of total chips (increased from 25%)
                self.model.addConstrs(
                    (self.x[casino, chip] <= 0.35 * total_chips for chip in self.high_value_chips),
                    name=f"high_value_max_35pct_{casino}"
                )
                
                # Modified constraint: $25 chip should be among the higher production (relaxed)
                # Instead of maximum, just ensure it's at least as much as $50 and $100 chips
                self.model.addConstr(
                    self.x[casino, "twenty-five"] >= self.x[casino, "fifty"],
                    name=f"twenty_five_vs_fifty_{casino}"
                )
                self.model.addConstr(
                    self.x[casino, "twenty-five"] >= self.x[casino, "hundred"],
                    name=f"twenty_five_vs_hundred_{casino}"
                )
            
            # NEW OBJECTIVE: maximize sum of logarithmic demand satisfaction (diminishing returns)
            # This creates diminishing marginal utility as production increases
            log_objective = gp.quicksum(
                self.log_vars[casino, chip] for casino in self.casinos for chip in self.chips
            )
            
            self.model.setObjective(log_objective, GRB.MAXIMIZE)
            
            print("Model built successfully with logarithmic demand objective!")
            print(f"Variables: {self.model.NumVars}")
            print(f"Constraints: {self.model.NumConstrs}")
            print(f"General Constraints: {self.model.NumGenConstrs}")
```

Finally, execute the new program and show logs.

### Prompt 7

Now introduce transportation costs from each factory to casino. Please ensure transportation costs from each factory to casino are evenly distributed. This involves adding transportation cost calculations and constraints to balance these costs across all factory-casino pairs.